In [1]:
import pandas as pd

crab_data = pd.read_csv('CrabAgePrediction.csv')
crab_data.tail()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
3888,F,1.4625,1.1375,0.3250,24.819987,11.651644,5.854172,6.378637,8
3889,F,1.5500,1.2125,0.4375,34.458817,15.450477,7.172423,9.780577,10
3890,I,0.6250,0.4625,0.1625,2.012815,0.765436,0.524466,0.637864,5
3891,I,1.0625,0.7750,0.2625,10.347568,4.507570,2.338834,2.976698,6
3892,I,0.7875,0.6125,0.2125,4.068153,1.502523,1.346601,1.417475,8


In [2]:
crab_data.columns

Index(['Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'Age'],
      dtype='object')

In [3]:
df = pd.get_dummies(crab_data, columns = ['Sex'])
df.columns

Index(['Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'Age', 'Sex_F', 'Sex_I', 'Sex_M'],
      dtype='object')

In [4]:
features = ['Length', 'Diameter', 'Height', 'Weight','Sex_F', 'Sex_I', 'Sex_M']
 
X = df[features]

y = df.Age

X.count()

Length      3893
Diameter    3893
Height      3893
Weight      3893
Sex_F       3893
Sex_I       3893
Sex_M       3893
dtype: int64

In [5]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

#scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#model details
model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(X.shape[1],)),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(1) 
])

model.compile(optimizer='adam', loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])

#train model
model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

#Save model to use in flask
model.save('my_model.keras')

#Save the scaler to use in flask
import joblib
joblib.dump(scaler, 'scaler.save')

Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 101.4534 - mean_absolute_error: 9.5725 - val_loss: 68.7749 - val_mean_absolute_error: 7.7026
Epoch 2/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 54.6763 - mean_absolute_error: 6.7642 - val_loss: 25.2773 - val_mean_absolute_error: 3.9931
Epoch 3/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 19.9558 - mean_absolute_error: 3.4698 - val_loss: 15.8445 - val_mean_absolute_error: 2.9209
Epoch 4/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12.8397 - mean_absolute_error: 2.7047 - val_loss: 12.2779 - val_mean_absolute_error: 2.5539
Epoch 5/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.3747 - mean_absolute_error: 2.3664 - val_loss: 9.9904 - val_mean_absolute_error: 2.2878
Epoch 6/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.3104 - mean_absolute_error: 2.1008 - val_loss: 8.3088 - val_mean_absolute_error: 2.0414
Epoch 7/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.2632 - mean_absolute_error: 1.8836 - val_loss: 7.5611

['scaler.save']

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

history_df = pd.DataFrame(history.history)

plt.figure(figsize=(10,6))
sns.lineplot(data=history_df[['loss', 'val_loss']])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend(labels=['Training Loss', 'Validation Loss'])
plt.show()

NameError: name 'history' is not defined